Created by Florent Poux. Licence CC

*   To reuse in your project, please cite the article.
*   Have fun with this notebook that you can very simply run (ctrl+Enter) !
*   The first time thought, it will ask you to get a key for it to be able to acces your Google drive folders if you want to work all remotely.
*   Simply accept, and then change the "10-MEDIUM/DATA/Point Cloud Sample/" by the folder path containing your data

Enjoy!

# Step 1: Setting up the environment

In [2]:
#http://www.open3d.org/docs/release/index.html
!pip install open3d

     |████████████████████████████████| 188.5 MB 88.3 MB/s eta 0:00:01   |██████████▊                     | 63.3 MB 5.0 MB/s eta 0:00:25     |███████████████████████▌        | 138.2 MB 86.8 MB/s eta 0:00:01████        | 141.6 MB 86.8 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 98.9 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 89.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 82.0 MB/s eta 0:00:01


  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=45767a06f3dc813428254449d7ffb138f47a4c7c6aa5742c542a2d00ba16b57a
  Stored in directory: /tmp/pip-ephem-wheel-cache-hfi4f40x/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


# Step 2: Load and prepare the data

In [1]:
#libraries used
import numpy as np
import open3d as o3d

In [7]:
#create paths and load data
scene = 'Barn'
model = 'pcd2'
# input_path=f"/scratch/ainaz/Tanks_and_Temples/{scene}/"
# output_path=f"/scratch/ainaz/Tanks_and_Temples/{scene}/"

# input_path=f"/scratch/ainaz/ETH3D/meadow/scan_clean/"
# output_path=f"/scratch/ainaz/ETH3D/meadow/scan_clean/"

input_path=f"/scratch/ainaz/RobotCarDataset/"
output_path=f"/scratch/ainaz/RobotCarDataset/"

dataname=f"{model}.ply"
# point_cloud= np.loadtxt(input_path+dataname,skiprows=1)
pcd = o3d.io.read_point_cloud(input_path+dataname)

In [8]:
#Format to open3d usable objects
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
# pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6]/255)
# pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,6:9])

In [9]:
print(pcd)

PointCloud with 486552 points.


In [10]:
pcd.estimate_normals()

In [60]:
o3d.io.write_point_cloud(output_path+'pcd2_n.ply', pcd)

True

# Step 3: Choose a meshing strategy

Now we are ready to start the surface reconstruction process by meshing the pcd point cloud. I will give my favorite way to efficiently obtain results, but before we dive in, some condensed details ar necessary to grasp the underlying processes. I will limit myself to two meshing strategies. See article

# Step 4: Process the data
## Strategy 1: BPA

In [11]:
#radius determination
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

In [ ]:
#computing the mehs
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))


In [ ]:
o3d.io.write_triangle_mesh(output_path + f"mesh2_bpa3.ply", bpa_mesh)

In [ ]:
#decimating the mesh
dec_mesh = mesh.simplify_quadric_decimation(100000)

*Optional ---*

In [ ]:
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

## Strategy 2: Poisson' reconstruction

In [47]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9, linear_fit=False)
print(poisson_mesh)

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 18839791 / 90286
[Open3D DEBUG] #   Got kernel density: 0.0400820 (s), 7244.61 (MB) / 10204.5 (MB) / 10987 (MB)
[Open3D DEBUG] #     Got normal field: 0.21438 (s), 7244.62 (MB) / 10204.5 (MB) / 10987 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.379018e-08 / 6.365999e-01
[Open3D DEBUG] #       Finalized tree: 0.874006 (s), 7244.62 (MB) / 10204.5 (MB) / 10987 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.620128 (s), 7244.62 (MB) / 10204.5 (MB) / 10987 (MB)
[Open3D DEBUG] #Set point constraints: 0.119959 (s), 7244.62 (MB) / 10204.5 (MB) / 10987 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 945351 / 1079824 / 577
[Open3D DEBUG] Memory Usage: 7244.621 MB
[Open3D DEBUG] # Linear system solved: 6.29285 (s), 7244.62 (MB) / 10204.5 (MB) / 10987 (MB)
[Open3D DEBUG] Got average: 0.049871 (s), 7244.62 (MB) / 10204.5 (MB) / 10987 (MB)
[Open3D DEBUG] Iso-Value: 4.999402e-01 = 9.418770e+06 / 1.883979e+07


In [48]:
#computing the mesh
# poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]

In [49]:
#cropping
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

# Step 5: Export and visualize

In [50]:
#export
# o3d.io.write_triangle_mesh(output_path+"bpa_mesh.ply", dec_mesh)
o3d.io.write_triangle_mesh(output_path + f"{model}_mesh.ply", p_mesh_crop)

[Open3D WARNING] Write Ply clamped color value to valid range


True

In [ ]:
#function creation
def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(path+"lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [ ]:
#execution of function
my_lods = lod_mesh_export(bpa_mesh, [100000,50000,10000,1000,100], ".ply", output_path)

In [ ]:
#execution of function
my_lods2 = lod_mesh_export(bpa_mesh, [8000,800,300], ".ply", output_path)